<a href="https://colab.research.google.com/github/andresenrique08/ML_2020/blob/master/Proyecto_Malware/Malware_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detección de Malware

**Curso de Machine Learning 2020-20**

*Proyecto Final*

Autores:


*   Andrés Enrique Ávila Rojas (201319127)
*   Jeison Alexander Timana Rangel (201617870)

Descripción de lo que se hace en el notebook...



In [1]:
# Importing libraries
import numpy as np
import pydrive as drive
from google.colab import drive
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('ggplot')

Se dan los permisos para acceder al almecenamiento de drive.

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


## Preparación de los datos

En esta sección se presenta la lectura y análisis preliminar de los datos de entrenamiento y los datos de validación. Estos datos vienen en dos archivos de formato csv separado por comas y pueden ser obtenidos de la página de la  <a href="https://www.kaggle.com/c/microsoft-malware-prediction/data">competencia de microsoft para la dección de malware</a> disponible en kaggle. El peso de los conjuntos de datos es:


*   train.csv: 4.08 GB
*   test.csv: 3.53 GB



**Datos de entrenamiento**

In [4]:
%%time
ruta_train = '/content/drive/MyDrive/Machine Learning/train.csv'
train  = pd.read_csv(ruta_train) 
train.head()

<string>:2: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 1min 27s, sys: 14.9 s, total: 1min 42s
Wall time: 1min 47s


In [6]:
train.shape

(8921483, 83)

**Datos de validación**

In [7]:
%%time
ruta_test = '/content/drive/MyDrive/Machine Learning/test.csv'
test  = pd.read_csv(ruta_test) 
test.head()

<string>:2: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 1min 21s, sys: 16.3 s, total: 1min 37s
Wall time: 1min 46s


In [8]:
test.shape

(7853253, 82)

Preliminarmente se puede analizar que los conjuntos de datos de entrenamiento y de validación no tienen la misma cantidad de variables (Como se observa en la siguiente tabla). Por ello se análizará inicialmente si todas las variables disponibles en el conjunto test, se encuentran en train, y cual variable no debería tenerse en cuenta debido a que no están en ambos conjuntos de datos.

Dataframe | No. de datos | No. de Variables |
----------|--------------|------------------|
Train | 8.921.483 | 83 |
Test | 7.853.253 | 82 |